In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import re

from tqdm import tqdm

In [76]:
import util.normalization as norm
import util.model as models
import util.utility as util

In [77]:
df_base = pd.read_csv("./mcd_result/base_mcd.csv")

df_base['aspect'] = df_base['aspect'].apply(lambda x: x.split(", "))
df_base['token_sentence'] = df_base['token_sentence'].apply(lambda x: x.split(".\n"))
df_base['token_lemma'] = df_base['token_lemma'].apply(lambda x: x.split(".\n"))
print(df_base.info())
df_base.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   reviewer_id     100 non-null    int64 
 1   review          100 non-null    object
 2   summarize       100 non-null    object
 3   aspect          100 non-null    object
 4   token_sentence  100 non-null    object
 5   token_lemma     100 non-null    object
 6   rating          100 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 5.6+ KB
None


,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"[someone spit, clear substance, everyone, norm...",[Why does it look like someone spit on my food...,"[why do it look like someone spit on my food, ...",1
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"[atmosphere, staff, food, mcdonalds]",[The staff at McDonalds are always friendly an...,[the staff at mcdonalds be always friendly and...,4
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"[refund, money, mobile order, line]","[Never got the refund in the app., Made a mobi...","[never get the refund in the app ., make a mob...",1
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"[customer, chicken sandwich, crispy chicken, c...",[Crispy chicken sandwich was customer service...,[crispy chicken sandwich be customer service b...,5
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","[order, large meal, close attention, english, ...","[I repeat my order 3 times in the drive thru, ...","[i repeat my order 3 time in the drive thru, i...",1


# Locating Aspcet per Sentence

In [58]:
def bullet_aspect(x):
    texts, array = x
    temp = {}
    for text in texts:
        pattern = re.compile(rf"{text}")
        
        idx_result = [idx for idx, element in enumerate(array) if pattern.search(element)]
        
        i = 0
        while i < len(idx_result):
            curr_idx = idx_result[i]
            if curr_idx not in temp.keys():
                temp[curr_idx] = text
            else:
                temp[curr_idx] = temp[curr_idx] + f', {text}'
            i += 1
    return list(temp.values())

# df_base = df_base.explode('aspect')
df_base['temp_1'] = list(zip(df_base['aspect'].values, df_base['token_lemma'].values))

tqdm.pandas()
df_base['aspect'] = df_base['temp_1'].progress_apply(bullet_aspect)


df_base.drop('temp_1', axis=1, inplace=True)

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 74804.78it/s]


In [59]:
df_base.head()

,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"[someone spit, clear substance, everyone, norm...",[Why does it look like someone spit on my food...,"[why do it look like someone spit on my food, ...",1
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"[atmosphere, food, staff, mcdonalds]",[The staff at McDonalds are always friendly an...,[the staff at mcdonalds be always friendly and...,4
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"[refund, refund, money, mobile order, line]","[Never got the refund in the app., Made a mobi...","[never get the refund in the app ., make a mob...",1
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"[customer, chicken sandwich, crispy chicken, c...",[Crispy chicken sandwich was customer service...,[crispy chicken sandwich be customer service b...,5
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","[order, order, close attention, large meal, la...","[I repeat my order 3 times in the drive thru, ...","[i repeat my order 3 time in the drive thru, i...",1


# Sentiment Score per Sentence

In [62]:
models.pattern_lexicon_model

def get_sentiment(x):
    temp = []
    for e in x:
        result = models.pattern_lexicon_model(e)
        temp.append(result)
    
    return temp


tqdm.pandas
df_base['sentiment_sentence'] = df_base['token_lemma'].apply(get_sentiment)

df_base.head()

,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,sentiment_sentence
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,"[someone spit, clear substance, everyone, norm...",[Why does it look like someone spit on my food...,"[why do it look like someone spit on my food, ...",1,"[negative, positive, negative, negative]"
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"[atmosphere, food, staff, mcdonalds]",[The staff at McDonalds are always friendly an...,[the staff at mcdonalds be always friendly and...,4,"[positive, positive]"
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,"[refund, refund, money, mobile order, line]","[Never got the refund in the app., Made a mobi...","[never get the refund in the app ., make a mob...",1,"[negative, negative, negative, negative, negat..."
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"[customer, chicken sandwich, crispy chicken, c...",[Crispy chicken sandwich was customer service...,[crispy chicken sandwich be customer service b...,5,"[negative, positive]"
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","[order, order, close attention, large meal, la...","[I repeat my order 3 times in the drive thru, ...","[i repeat my order 3 time in the drive thru, i...",1,"[negative, positive, negative, negative]"


In [71]:
df_base['token_lemma'].iloc[3]

['crispy chicken sandwich be customer service be quick and p.m .',
 'customer service be quick and easy .']

In [70]:
df_base['aspect'].iloc[3]

['customer, chicken sandwich, crispy chicken, customer service',
 'customer, customer service']

In [73]:
df_base['aspect'] = df_base['aspect'].apply(lambda x: "\n ".join([f"• {item}" for item in x]))
df_base['sentiment_sentence'] = df_base['sentiment_sentence'].apply(lambda x: "\n ".join([f"• {item}" for item in x]))
df_base['token_sentence'] = df_base['token_sentence'].apply(lambda x: "\n ".join([f"• {item}" for item in x]))

df_base.head()

,reviewer_id,review,summarize,aspect,token_sentence,token_lemma,rating,sentiment_sentence
0,1,Why does it look like someone spit on my food?...,Why does it look like someone spit on my food...,• someone spit\n • clear substance\n • everyon...,• Why does it look like someone spit on my foo...,"[why do it look like someone spit on my food, ...",1,• negative\n • positive\n • negative\n • negative
1,2,It'd McDonalds. It is what it is as far as the...,The staff at McDonalds are always friendly an...,"• atmosphere, food\n • staff, mcdonalds",• The staff at McDonalds are always friendly a...,[the staff at mcdonalds be always friendly and...,4,• positive\n • positive
2,3,Made a mobile order got to the speaker and che...,Never got the refund in the app. Made a mobil...,• refund\n • refund\n • money\n • mobile order...,• Never got the refund in the app.\n • Made a ...,"[never get the refund in the app ., make a mob...",1,• negative\n • negative\n • negative\n • negat...
3,4,My mc. Crispy chicken sandwich was ÃÂ¯ÃÂ¿ÃÂ...,My mc. Crispy chicken sandwich was customer ...,"• customer, chicken sandwich, crispy chicken, ...",• Crispy chicken sandwich was customer servic...,[crispy chicken sandwich be customer service b...,5,• negative\n • positive
4,5,"I repeat my order 3 times in the drive thru, a...","I repeat my order 3 times in the drive thru, ...","• order\n • order, close attention\n • large m...",• I repeat my order 3 times in the drive thru\...,"[i repeat my order 3 time in the drive thru, i...",1,• negative\n • positive\n • negative\n • negative


In [74]:
df_base[['reviewer_id', 'review', 'summarize', 
         'aspect', 'token_sentence', 'sentiment_sentence', 'rating']]\
        .to_csv("./mcd_result/helper_mcd.csv", index=False)